# PUMS Household Income vs. AMI (2010) in Phoenix  PUMAS

- https://www.census.gov/data/developers/data-sets/

For households by income and household size to compare to HUD AMI in same year
-  https://api.census.gov/data/2021/acs/acs1/pums/variables.html

2010 AMI by HH Size (from State of Arizona)
- https://housing.az.gov/sites/default/files/documents/files/2010_HOME-HTF_Rent-Income%2520Limits.pdf

https://www.huduser.gov/portal/datasets/il/il10/AZ.pdf

In [1]:
import pandas as pd
import math
import numpy as np
import os

In [2]:
import get_pums as get
import pums as calc

In [3]:
#Search parameters
y1 = '2010'

sample = 'acs1'

phx_pumas = ['0400113','0400114','0400115','0400116','0400117',\
             '0400118','0400119','0400120','0400121','0400122','0400123',\
             '0400125','0400128','0400112','0400129']

#north_pumas = ['0400112','0400129']

data_cols = 'SERIALNO,ST,PUMA,HINCP,NP,WGTP,ADJINC,ADJHSG,GRNTP'

ADJHSG - adjustment factor for housing dollar amounts (6 decimal places)
1000000 = 1.000000

FHINCP - income flag - 1: yes | https://api.census.gov/data/2021/acs/acs1/pums/variables/FHINCP.json

HINCP - HHI in past 12 months (not -60000:n/a, 0:no income, -59999 loss of 59k+)
-1 to -59998 loss, 1+
https://api.census.gov/data/2021/acs/acs1/pums/variables/HINCP.json

GRNTP - gross rent (monthly) (not 0: n/a,not paying rent, owned)
https://api.census.gov/data/2021/acs/acs1/pums/variables/GRNTP.json

In [4]:
#AMI bands in 2010 by household size
AMI_30pct = {'1':14000,'2':16000,'3':18000,'4':20000,'5':21600,'6':23200,\
            '7':24800,'8':26400}
AMI_50pct = {'1':23350,'2':26650,'3':30000,'4':33300,'5':36000,'6':38650,\
            '7':41300,'8':44000}
AMI_80pct = {'1':37350,'2':42650,'3':48000,'4':53300,'5':57600,'6':61850,\
            '7':66100,'8':70400}
AMI_100pct = {'1':46700,'2':53300,'3':60000,'4':66600,'5':72000,'6':77300,\
            '7':82600,'8':88300}

In [5]:
#Housing costs affordable to different AMI bands based on household size
aff_1p = {'30pct':385,'50pct':642,'80pct':1027,'100pct':1284}
aff_2p = {'30pct':440,'50pct':733,'80pct':1173,'100pct':1466}
aff_3p = {'30pct':495,'50pct':825,'80pct':1320,'100pct':1650}
aff_4p = {'30pct':550,'50pct':916,'80pct':1466,'100pct':1832}
aff_5p = {'30pct':594,'50pct':990,'80pct':1584,'100pct':1980}
aff_6p = {'30pct':638,'50pct':1063,'80pct':1693,'100pct':2126}
aff_7p = {'30pct':682,'50pct':1136,'80pct':1818,'100pct':2272}
aff_8p = {'30pct':726,'50pct':1210,'80pct':2272,'100pct':2420}

unit_afford = {'1':[0,365,642,1027,1284,1000000000],\
              '2':[0,440,733,1173,1466,1000000000],\
              '3':[0,495,825,1320,1650,1000000000],\
              '4':[0,550,916,1466,1832,1000000000],\
              '5':[0,594,990,1584,1980,1000000000],\
              '6':[0,638,1063,1693,2126,1000000000],\
              '7':[0,682,1136,1818,2272,1000000000],\
              '8':[0,726,1210,1936,2420,1000000000]}

inc_lbls = ['u30_ami','30_50_ami','50_80_ami','80_100_ami','o100_ami']

In [6]:
# create a list of replicate weights
repwt = 'WGTP'
repwts = [repwt+str(i) for i in range(1, 81)]

## Get PUMA data

In [7]:
df = get.get_puma(sample,y1,data_cols)

In [8]:
df.head()

,SERIALNO,ST,PUMA,HINCP,NP,WGTP,ADJINC,ADJHSG,GRNTP,WGTP1,...,WGTP71,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80
0,70,4,300,5200,2,43,1007624,1000000,0,42,...,13,86,39,80,74,40,48,38,46,14
1,328,4,600,15000,1,207,1007624,1000000,0,177,...,186,341,84,295,190,99,91,68,191,321
2,881,4,120,194000,2,57,1007624,1000000,0,57,...,90,59,54,57,16,55,55,101,17,59
3,1010,4,120,35000,1,167,1007624,1000000,995,48,...,173,292,50,160,167,46,163,172,49,292
4,1103,4,207,13800,1,64,1007624,1000000,0,19,...,64,62,109,21,105,66,20,19,19,19


In [9]:
df['GEO_ID'] = df['ST'].astype(str).str.zfill(2)+df['PUMA'].astype(str).str.zfill(5)
df = df[df.GEO_ID.isin(phx_pumas)]
df  = df.drop(['SERIALNO','ST','PUMA'],axis=1)
df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']] #move id to first col
for col in df.columns[1:]: df[col] = df[col].astype(float)

In [10]:
df['HHSz'] = pd.cut(df['NP'],bins=[0,1,2,3,4,5,6,7,20],\
                   labels=['1','2','3','4','5','6','7','8'])
df['HHSz'] = df['HHSz'].astype(str)

In [11]:
dff = df[~(df.HHSz.isna()) & (df.HINCP!=-60000)&(df.HINCP!=0)&(df.GRNTP!=0)].copy()
dff['HINCP'] = (dff.ADJINC/1000000) * dff.HINCP

In [12]:
dff['hou_cost'] = dff.GRNTP
dff['aff_cost'] = np.where(dff.HINCP>=1,(dff.HINCP*0.333)/12,0)

In [13]:
#assign AMI range based on household size and ami dictionaries
dff['AMI_range'] = np.where((dff['HINCP']<=dff['HHSz'].map(AMI_30pct)),'u30_ami',\
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_30pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_50pct)),'30_50_ami',\
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_50pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_80pct)),\
                   '50_80_ami',np.where((dff['HINCP']>dff['HHSz'].map(AMI_80pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_100pct)),\
                   '80_100_ami','o100_ami'))))

In [14]:
dff['cost_burdened'] = np.where(dff.hou_cost>dff.aff_cost,'burdened','not burdened')

In [15]:
#who is the unit affordable to based on the rent
dff['unit_aff'] = np.where(dff.HHSz=='1',pd.cut(dff['hou_cost'],bins=unit_afford['1'],labels=inc_lbls),\
                  np.where(dff.HHSz=='2',pd.cut(dff['hou_cost'],bins=unit_afford['2'],labels=inc_lbls),\
                  np.where(dff.HHSz=='3',pd.cut(dff['hou_cost'],bins=unit_afford['3'],labels=inc_lbls),\
                  np.where(dff.HHSz=='4',pd.cut(dff['hou_cost'],bins=unit_afford['4'],labels=inc_lbls),\
                  np.where(dff.HHSz=='5',pd.cut(dff['hou_cost'],bins=unit_afford['5'],labels=inc_lbls),\
                  np.where(dff.HHSz=='6',pd.cut(dff['hou_cost'],bins=unit_afford['6'],labels=inc_lbls),\
                  np.where(dff.HHSz=='7',pd.cut(dff['hou_cost'],bins=unit_afford['7'],labels=inc_lbls),\
                  np.where(dff.HHSz=='8',pd.cut(dff['hou_cost'],bins=unit_afford['8'],labels=inc_lbls),''))))))))

In [16]:
dff.head(3)

,GEO_ID,HINCP,NP,WGTP,ADJINC,ADJHSG,GRNTP,WGTP1,WGTP2,WGTP3,...,WGTP77,WGTP78,WGTP79,WGTP80,HHSz,hou_cost,aff_cost,AMI_range,cost_burdened,unit_aff
3,0400120,35266.84000,1.0,167.0,1007624.0,1000000.0,995.0,48.0,284.0,50.0,...,163.0,172.0,49.0,292.0,1,995.0,978.654810,50_80_ami,burdened,50_80_ami
7,0400112,12494.53760,1.0,94.0,1007624.0,1000000.0,470.0,97.0,94.0,170.0,...,28.0,24.0,84.0,96.0,1,470.0,346.723418,u30_ami,burdened,30_50_ami
13,0400122,25513.03968,5.0,84.0,1007624.0,1000000.0,1518.0,25.0,86.0,135.0,...,25.0,24.0,88.0,25.0,5,1518.0,707.986851,30_50_ami,burdened,50_80_ami


### table by PUMA for renters by AMI range - cost burdened vs. not cost burdened

In [17]:
def make_est(df):
    df['hh_SE'] = df.apply(lambda x: (calc.get_se(x['WGTP'],x[repwts])),axis=1)
    df['hh_MOE'] = df.apply(lambda x: (calc.get_moe(x['hh_SE'])),axis=1)
    df['hh_CV'] = df.apply(lambda x: (calc.get_cv(x['WGTP'],x['hh_SE'])),axis=1)
    df.rename(columns={'WGTP':'hh'},inplace=True)
    return df

In [18]:
drop_cols = ['HINCP','NP','ADJINC','ADJHSG','GRNTP','hou_cost', 'aff_cost','HHSz']

In [19]:
table = dff.copy().drop(columns=drop_cols)

In [20]:
table = table.groupby(['GEO_ID','AMI_range','unit_aff','cost_burdened']).sum().reset_index()

In [21]:
table_2 = table.copy().groupby(['GEO_ID','AMI_range','unit_aff','cost_burdened']).sum().reset_index()
table_2 = make_est(table_2)
table_2 = table_2.drop(columns=repwts)

In [22]:
rent_ami = table.copy().drop(columns=['unit_aff','cost_burdened']).groupby(['GEO_ID','AMI_range']).sum().reset_index()
rent_ami = make_est(rent_ami)
rent_ami = rent_ami.drop(columns=repwts)

In [23]:
rent_aff = table.copy().drop(columns=['AMI_range','cost_burdened']).groupby(['GEO_ID','unit_aff']).sum().reset_index()
rent_aff = make_est(rent_aff)
rent_aff = rent_aff.drop(columns=repwts)

In [24]:
cost_burd = table.copy().drop(columns=['unit_aff','AMI_range']).groupby(['GEO_ID','cost_burdened']).sum().reset_index()
cost_burd = make_est(cost_burd)
cost_burd = cost_burd.drop(columns=repwts)

In [25]:
cost_burd_ami = table.copy().drop(columns=['unit_aff']).groupby(['GEO_ID','cost_burdened','AMI_range']).sum().reset_index()
cost_burd_ami = make_est(cost_burd_ami)
cost_burd_ami = cost_burd_ami.drop(columns=repwts)

In [26]:
table_3 = table.copy().drop(columns='cost_burdened').groupby(['GEO_ID','AMI_range','unit_aff']).sum().reset_index()
table_3 = make_est(table_3)
table_3 = table_3.drop(columns=repwts)

In [27]:
table_2 = pd.pivot_table(table_2,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['AMI_range','unit_aff','cost_burdened'],aggfunc=np.sum).reset_index()

rent_ami = pd.pivot_table(rent_ami,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['AMI_range'],aggfunc=np.sum).reset_index()

rent_aff = pd.pivot_table(rent_aff,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['unit_aff'],aggfunc=np.sum).reset_index()

cost_burd = pd.pivot_table(cost_burd,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['cost_burdened'],aggfunc=np.sum).reset_index()

cost_burd_ami = pd.pivot_table(cost_burd_ami,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['AMI_range','cost_burdened'],aggfunc=np.sum).reset_index()

In [28]:
table_3 = pd.pivot_table(table_3,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['AMI_range','unit_aff'],aggfunc=np.sum).reset_index()

In [29]:
with pd.ExcelWriter(f'output/pums_grossrent_gap_2010.xlsx') as writer:
    table_2.to_excel(writer,sheet_name='full_renter')
    rent_ami.to_excel(writer,sheet_name='hh_ami_lvl_renter')
    rent_aff.to_excel(writer,sheet_name='hh_afford_byAMI_renter')
    cost_burd.to_excel(writer,sheet_name='cost_burdened_renter')
    cost_burd_ami.to_excel(writer,sheet_name='cost_burdened_byAMI_renter')
    table_3.to_excel(writer,sheet_name='demand_supply_amilvl')